In [ ]:
!pip install cdsapi numpy urllib3  pandas datetime xarray[complete] 

In [21]:
import xarray as xr
# plt.style.available
#import colormaps as cmaps 
#import concurrent
import numpy as np
# # Libraries to assist with animation and visualisations
# %matplotlib inline
# import matplotlib.pyplot as plt
# from matplotlib import animation
from IPython.display import HTML

# Disable warnings for data download via API
import urllib3
urllib3.disable_warnings()

import json
import pandas as pd
from datetime import datetime, timedelta
import xarray as xr
# import cfgrib
# import eccodes
# import geopandas as gpd
# from shapely.geometry import Point
import zipfile
import os
import logging

from zipfile import ZipFile 
import cdsapi
URL = 'https://ads.atmosphere.copernicus.eu/api'
KEY = '87bb8d36-2458-48c2-b5e3-f1fd352291bc'


In [22]:

logging.basicConfig(level=logging.INFO)

def dict_l(data, fm):
    try:
        # Validate filename
       
        
        # Convert data to dictionary
        logging.info("Converting data to dictionary...")
        data_dict = data.squeeze().to_dict(data='list', encoding=False)
        
        # Helper function to recursively round numeric values
        def round_numbers(obj):
            if isinstance(obj, dict):
                return {k: round_numbers(v) for k, v in obj.items()}
            elif isinstance(obj, list):
                return [round_numbers(item) for item in obj]
            elif isinstance(obj, (float, np.float32, np.float64)):
                return round(float(obj), 2)  # Round floats to 2 decimal places
            elif isinstance(obj, (int, np.int32, np.int64)):
                return int(obj)  # Keep integers as-is
            else:
                return obj  # Leave other types unchanged
        
        # Apply rounding to the dictionary
        logging.info("Rounding numeric values to 2 decimal places...")
        rounded_data_dict = round_numbers(data_dict)
        
        # Helper function to handle non-serializable types
        def convert_types(obj):
            if isinstance(obj, dict):
                return {k: convert_types(v) for k, v in obj.items()}
            elif isinstance(obj, list):
                return [convert_types(item) for item in obj]
            elif isinstance(obj, (np.datetime64, pd.Timestamp, datetime)):
                return obj.isoformat()
            elif isinstance(obj, timedelta):
                return obj.total_seconds()
            elif isinstance(obj, np.ndarray):
                return obj.tolist()
            elif isinstance(obj, (np.float32, np.float64)):
                return float(obj)
            elif isinstance(obj, (np.int32, np.int64)):
                return int(obj)
            raise TypeError(f"Type {type(obj)} not serializable")
        
        # Save to JSON file
        file = os.path.join(os.getcwd(), f"{fm}.json")
        logging.info(f"Saving data to {file}...")
        with open(file, 'w') as json_file:
            json.dump(rounded_data_dict, json_file, indent=4, default=convert_types)
        
        logging.info("Data successfully saved.")
    
    except Exception as e:
        logging.error(f"An error occurred: {e}")
        raise

In [ ]:
from datetime import date,timedelta
time1=(date.today()).strftime("%Y-%m-%d")
time1


In [ ]:

dataset = "cams-global-atmospheric-composition-forecasts"
lead= [str(i) for i in range(0,73)]

request = {
    "date": [f"{time1}/{time1}"],
    "time": ["00:00"],
    "leadtime_hour": lead,
    "type": ["forecast"],
    "data_format": "netcdf_zip",
    "variable": [
         "2m_dewpoint_temperature",
        "dust_aerosol_optical_depth_550nm",
        "mean_sea_level_pressure",
        "nitrate_aerosol_optical_depth_550nm",
        "particulate_matter_2.5um",
        "particulate_matter_10um",
        "sea_salt_aerosol_optical_depth_550nm",
        "sulphate_aerosol_optical_depth_550nm",
        "total_aerosol_optical_depth_550nm",
        "total_column_water_vapour",
        "convective_precipitation",
        "high_cloud_cover",
        "low_cloud_cover",
        "medium_cloud_cover",
        "total_cloud_cover",
        "most_unstable_convective_available_potential_energy",
        "most_unstable_convective_inhibition"

    ],
    "area": [50, 50, 0, 100]
}

client = cdsapi.Client(url=URL, key=KEY)
client.retrieve(dataset, request).download('test.zip')


In [14]:
ls

 Volume in drive E is New Volume
 Volume Serial Number is 1C64-8838

 Directory of e:\test_climate\subhrajitjubu.github.io\srcc

27-02-2025  18:09    <DIR>          .
27-02-2025  12:02    <DIR>          ..
27-02-2025  12:02        44,257,849 2m_temp.json
27-02-2025  12:02        41,211,505 AOD_DUST.json
27-02-2025  12:02        39,982,138 AOD_NIT.json
27-02-2025  12:02        41,601,823 AOD_SEA.json
27-02-2025  12:02        46,274,024 AOD_SUL.json
27-02-2025  12:02        45,449,652 AOD_TOT.json
27-02-2025  12:02        36,880,956 CAPE.json
27-02-2025  12:02        36,794,915 CIN.json
27-02-2025  12:02        27,281,796 CP.json
27-02-2025  12:02        43,723,968 D2M.json
27-02-2025  18:09                 0 data_sfc.nc
27-02-2025  17:37            10,950 geoheatmap.html
27-02-2025  18:07            15,529 geomap_multiple.html
27-02-2025  12:02        36,021,840 HCC.json
27-02-2025  17:46            18,781 heatmap.js
27-02-2025  12:02         5,469,912 IND.json
27-02-2025  12:02        

In [15]:

with ZipFile("test.zip", 'r') as zObject: 
    zObject.extractall(path="./extracted_files")  # Specify a different path for extraction

ds = xr.open_dataset('./extracted_files/data_sfc.nc')  # Update the path to the extracted file

In [16]:
ds.valid_time.values

array([['2025-02-27T00:00:00.000000000', '2025-02-27T01:00:00.000000000',
        '2025-02-27T02:00:00.000000000', '2025-02-27T03:00:00.000000000',
        '2025-02-27T04:00:00.000000000', '2025-02-27T05:00:00.000000000',
        '2025-02-27T06:00:00.000000000', '2025-02-27T07:00:00.000000000',
        '2025-02-27T08:00:00.000000000', '2025-02-27T09:00:00.000000000',
        '2025-02-27T10:00:00.000000000', '2025-02-27T11:00:00.000000000',
        '2025-02-27T12:00:00.000000000', '2025-02-27T13:00:00.000000000',
        '2025-02-27T14:00:00.000000000', '2025-02-27T15:00:00.000000000',
        '2025-02-27T16:00:00.000000000', '2025-02-27T17:00:00.000000000',
        '2025-02-27T18:00:00.000000000', '2025-02-27T19:00:00.000000000',
        '2025-02-27T20:00:00.000000000', '2025-02-27T21:00:00.000000000',
        '2025-02-27T22:00:00.000000000', '2025-02-27T23:00:00.000000000',
        '2025-02-28T00:00:00.000000000', '2025-02-28T01:00:00.000000000',
        '2025-02-28T02:00:00.000000000

In [ ]:
aod_du=ds.sel(longitude=slice(50,100),latitude=slice(40,0))['duaod550']
aod_ss=ds.sel(longitude=slice(50,100),latitude=slice(40,0))['ssaod550']
aod_su=ds.sel(longitude=slice(50,100),latitude=slice(40,0))['suaod550']
aod=ds.sel(longitude=slice(50,100),latitude=slice(40,0))['aod550']
aod_ni=ds.sel(longitude=slice(50,100),latitude=slice(40,0))['niaod550']
time=ds.valid_time





pm25=ds.sel(longitude=slice(50,100),latitude=slice(40,0))['pm2p5']
pm10=ds.sel(longitude=slice(50,100),latitude=slice(40,0))['pm10']
t2=ds.sel(longitude=slice(50,100),latitude=slice(40,0))['t2m']

lcc=ds.sel(longitude=slice(50,100),latitude=slice(40,0))['lcc']
mcc=ds.sel(longitude=slice(50,100),latitude=slice(40,0))['mcc']
hcc=ds.sel(longitude=slice(50,100),latitude=slice(40,0))['hcc']
tcc=ds.sel(longitude=slice(50,100),latitude=slice(40,0))['tcc']

cape=ds.sel(longitude=slice(50,100),latitude=slice(40,0))['mucape']
cin=ds.sel(longitude=slice(50,100),latitude=slice(40,0))['mucin']

d2m=ds.sel(longitude=slice(50,100),latitude=slice(40,0))['d2m']
cp=ds.sel(longitude=slice(50,100),latitude=slice(40,0))['cp']




In [24]:
aod_ss

<xarray.DataArray 'ssaod550' (forecast_period: 73, forecast_reference_time: 1,
                              latitude: 100, longitude: 126)> Size: 4MB
[919800 values with dtype=float32]
Coordinates:
  * forecast_period          (forecast_period) timedelta64[ns] 584B 00:00:00 ...
  * forecast_reference_time  (forecast_reference_time) datetime64[ns] 8B 2025...
  * latitude                 (latitude) float64 800B 39.6 39.2 38.8 ... 0.4 0.0
  * longitude                (longitude) float64 1kB 50.0 50.4 ... 99.6 100.0
    valid_time               (forecast_reference_time, forecast_period) datetime64[ns] 584B ...
Attributes: (12/33)
    GRIB_paramId:                             210208
    GRIB_dataType:                            fc
    GRIB_numberOfPoints:                      15876
    GRIB_typeOfLevel:                         surface
    GRIB_stepUnits:                           1
    GRIB_stepType:                            instant
    ...                                       ...
    GRIB_units:                               ~
    long_name:                                Sea Salt Aerosol Optical Depth ...
    units:                                    ~
    standard_name:                            unknown
    GRIB_number:                              0
    GRIB_surface:                             0.0

In [ ]:

 dict_l(1e9*pm25,"PM25")
 dict_l(1e9*pm10,"PM10")

 dict_l(lcc,"LCC")
 dict_l(mcc,"MCC")
 dict_l(hcc,"HCC")
 dict_l(tcc,"TCC")

 dict_l(cape,"CAPE")
 dict_l(cin,"CIN")
 dict_l(d2m-273.15,"D2M")
 dict_l(cp*1000,"CP")

 dict_l(aod_ss,"AOD_SEA")
 dict_l(aod_su,"AOD_SUL")
 dict_l(aod_ni,"AOD_NIT")
 dict_l(aod_du,"AOD_DUST")
 dict_l(aod,"AOD_TOT")

dict_l(time,"TIME")






INFO:root:Converting data to dictionary...
INFO:root:Rounding numeric values to 2 decimal places...
INFO:root:Saving data to e:\test_climate\subhrajitjubu.github.io\srcc\2m_temp.json...
INFO:root:Data successfully saved.
INFO:root:Converting data to dictionary...
INFO:root:Rounding numeric values to 2 decimal places...
INFO:root:Saving data to e:\test_climate\subhrajitjubu.github.io\srcc\rf.json...
INFO:root:Data successfully saved.
INFO:root:Converting data to dictionary...
INFO:root:Rounding numeric values to 2 decimal places...
INFO:root:Saving data to e:\test_climate\subhrajitjubu.github.io\srcc\msl.json...
INFO:root:Data successfully saved.
INFO:root:Converting data to dictionary...
INFO:root:Rounding numeric values to 2 decimal places...
INFO:root:Saving data to e:\test_climate\subhrajitjubu.github.io\srcc\tcwv.json...
INFO:root:Data successfully saved.
INFO:root:Converting data to dictionary...
INFO:root:Rounding numeric values to 2 decimal places...
INFO:root:Saving data to e:\